In [1]:
from sklearn.cluster import  MiniBatchKMeans, KMeans, DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import pandas as pd
import math
from pylab import *
from scipy.spatial.distance import cdist
from mpl_toolkits.mplot3d import Axes3D

In [2]:
from logistics_functions import *
from logistics_model import logistics_model

In [3]:
n_regen =20
n_collection = 1

In [ ]:
Logistics = logistics_model('..\shapefiles\\building_block_people_WGS_2.csv', n_regen, n_collection)

In [ ]:
distance_regeneration, distance_collection = Logistics.logistics_distances()

In [15]:
distance_regeneration

,cluster,num_people,total_dist_m
0,0,26924,6.229376e+06
1,1,41393,1.123000e+08
2,2,53575,8.453394e+07
3,3,40722,9.578943e+07
4,4,31650,6.845819e+07
5,5,43110,6.751372e+07
6,6,38107,7.436272e+07
7,7,31017,6.292407e+07
8,8,67531,2.012356e+07
9,9,27948,9.615675e+07
